In [1]:
import click
import torch
import logging
import random
import numpy as np

from utils.config import Config
from utils.visualization.plot_images_grid import plot_images_grid
from DeepSAD import DeepSAD
from datasets.main import load_dataset

In [3]:
# Get configuration
cfg = Config(locals().copy())
xp_path = "../log"
data_path = "../data"
dataset_name = 'cifar10'
normal_class = 0
ratio_known_outlier = 0.01
ratio_known_normal = 0
ratio_pollution = 0.1
n_known_outlier_classes = 9
known_outlier_class = 1
net_name = 'ResNet_18'
load_config = None
num_threads = 0
n_jobs_dataloader = 0
seed = 0 
eta = 1.0
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
log_file = xp_path + '/log.txt'
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Print paths
logger.info('Log file is %s' % log_file)
logger.info('Data path is %s' % data_path)
logger.info('Export path is %s' % xp_path)

# Print experimental setup
logger.info('Dataset: %s' % dataset_name)
logger.info('Normal class: %d' % normal_class)
logger.info('Ratio of labeled normal train samples: %.2f' % ratio_known_normal)
logger.info('Ratio of labeled anomalous samples: %.2f' % ratio_known_outlier)
logger.info('Pollution ratio of unlabeled train data: %.2f' % ratio_pollution)
if n_known_outlier_classes == 1:
    logger.info('Known anomaly class: %d' % known_outlier_class)
else:
    logger.info('Number of known anomaly classes: %d' % n_known_outlier_classes)
    logger.info('Network: %s' % net_name)

# If specified, load experiment config from JSON-file
if load_config:
    cfg.load_config(import_json=load_config)
    logger.info('Loaded configuration from %s.' % load_config)

# Print model configuration
logger.info('Eta-parameter: %.2f' % eta)



# Default device to 'cpu' if cuda is not available
device = 'cuda'
if not torch.cuda.is_available():
    device = 'cpu'
# Set the number of threads used for parallelizing CPU operations
if num_threads > 0:
    torch.set_num_threads(num_threads)
    logger.info('Computation device: %s' % device)
    logger.info('Number of threads: %d' % num_threads)
    logger.info('Number of dataloader workers: %d' % n_jobs_dataloader)
# Load data
dataset = load_dataset(dataset_name, data_path, normal_class, known_outlier_class, n_known_outlier_classes,
                        ratio_known_normal, ratio_known_outlier, ratio_pollution)


INFO:root:Log file is ../log/log.txt
INFO:root:Data path is ../data
INFO:root:Export path is ../log
INFO:root:Dataset: cifar10
INFO:root:Normal class: 0
INFO:root:Ratio of labeled normal train samples: 0.00
INFO:root:Ratio of labeled anomalous samples: 0.01
INFO:root:Pollution ratio of unlabeled train data: 0.10
INFO:root:Number of known anomaly classes: 9
INFO:root:Network: ResNet_18
INFO:root:Eta-parameter: 1.00


Files already downloaded and verified
Files already downloaded and verified


In [13]:
print(dataset)

NameError: name 'dataset' is not defined

In [8]:
# Log random sample of known anomaly classes if more than 1 class
if n_known_outlier_classes > 1:
    logger.info('Known anomaly classes: %s' % (dataset.known_outlier_classes,))

# Initialize DeepSAD model and set neural network phi
deepSAD = DeepSAD(eta)
deepSAD.set_network(net_name)

INFO:root:Known anomaly classes: (7, 9, 1, 3, 5, 4, 2, 6, 8)


In [12]:
ae_optimizer_name = "adam"
ae_lr = 0.0001
ae_n_epochs = 100
ae_lr_milestone = [20, 50, 75]
ae_batch_size = 128
ae_weight_decay = 1e-6
logger.info('Pretraining optimizer: %s' % ae_optimizer_name)
logger.info('Pretraining learning rate: %g' % ae_lr)
logger.info('Pretraining epochs: %d' % ae_n_epochs)
logger.info('Pretraining learning rate scheduler milestones: %s' % (ae_lr_milestone,))
logger.info('Pretraining batch size: %d' % ae_batch_size)
logger.info('Pretraining weight decay: %g' % ae_weight_decay)

# Pretrain model on dataset (via autoencoder)
deepSAD.pretrain(dataset,
                    optimizer_name=ae_optimizer_name,
                    lr= ae_lr,
                    n_epochs=ae_n_epochs,
                    lr_milestones=ae_lr_milestone,
                    batch_size=ae_batch_size,
                    weight_decay=ae_weight_decay,
                    device=device,
                    n_jobs_dataloader=n_jobs_dataloader)

# Save pretraining results
deepSAD.save_ae_results(export_json=xp_path + '/ae_results.json')

INFO:root:Pretraining optimizer: adam
INFO:root:Pretraining learning rate: 0.0001
INFO:root:Pretraining epochs: 100
INFO:root:Pretraining learning rate scheduler milestones: [20, 50, 75]
INFO:root:Pretraining batch size: 128
INFO:root:Pretraining weight decay: 1e-06
INFO:root:Starting pretraining...


TypeError: Cannot handle this data type: (1, 1, 32), <f4